In [4]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [22]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-12 05:36:52--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.00MB/s    in 0.2s    

2023-01-12 05:36:53 (5.00 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenges").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [24]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [25]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
books_df = df
books_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [26]:
# Vine_table DataFrame. 
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28HBXXO1UEVJT|          5|            0|          0|   N|                N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|                Y|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|                N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|                Y|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|                N|
| RCLZ5OKZNUSY4|          5|            0|          0|   N|                Y|
|R1S65DJYEI89G4|          4|            8|         17|   N|                N|
|R3KQYBQOLYDETV|          4|            2|          2|   N|                N|
|R3QV8K7CSU8K2W|          5|            0|          0|   N|                N|
|R3W5A1WUGO5VQ0|          4|            0|          1|   N|     

In [27]:
# total_votes_count DataFrame.
total_votes_count = vine_df.filter( vine_df.total_votes  >= "20") 
total_votes_count.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18VIM840CEFRP|          1|           16|        105|   N|                N|
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
| RW00TDPV9U93E|          1|           35|         76|   N|                N|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R18I0XBYWFSR5T|          1|           16|         50|   N|                N|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|     

In [28]:
# new_votes DataFrame.
new_votes_df = total_votes_count.filter((total_votes_count.helpful_votes / total_votes_count.total_votes) >= .5)
new_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|
|R34ZTQTHWRHT5R|          5|           18|         20|   N|                Y|
|R367X555006ROC|          1|           39|         44|   N|                N|
| R4NCMAZCRAXHT|          3|           19|         21|   N|     

In [29]:
# unpaid_program_review DataFrame.
unpaid_program_review = new_votes_df.filter(new_votes_df.vine == "N")
unpaid_program_review.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|
|R34ZTQTHWRHT5R|          5|           18|         20|   N|                Y|
|R367X555006ROC|          1|           39|         44|   N|                N|
| R4NCMAZCRAXHT|          3|           19|         21|   N|     

In [30]:
# paid_program_review DataFrame.
paid_program_review = new_votes_df.filter(new_votes_df.vine == "Y")
paid_program_review.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2Z8083V8ZFQVZ|          3|           82|         87|   Y|                N|
| RK25TAO4GGS5G|          3|           22|         24|   Y|                N|
|R2SJQG3C6KY0M8|          3|           18|         20|   Y|                N|
|R12CHC0CB2WASU|          5|           43|         48|   Y|                N|
|R2RY328TIDXMTE|          2|           50|         61|   Y|                N|
|R3K8OQU4PEP8W7|          4|           40|         46|   Y|                N|
|R1JI1A7JT99538|          3|           28|         40|   Y|                N|
|R247K3K5SVPJUL|          3|           15|         20|   Y|                N|
|R27UEUDUYPLXZC|          5|           20|         24|   Y|                N|
|R2QQDNBYDIEL9F|          5|           35|         41|   Y|     

In [31]:
# Total reviews.
total_reviews = new_votes_df.count()
total_reviews

740983

In [32]:
# Total number of the unpaid program reviews
total_unpaid = unpaid_program_review.count()
total_unpaid

736202

In [33]:
# Total number of the paid program reviews
total_paid = paid_program_review.count()
total_paid

4781

In [34]:
# Number of 5 star reviews 
five_star_total = new_votes_df.filter(new_votes_df.star_rating == 5).count()
five_star_total

413293

In [35]:
# Number of 5 star reviews for paid
five_star_paid = paid_program_review.filter(paid_program_review.star_rating == 5).count()
five_star_paid

1604

In [36]:
# Number of 5 star reviews for unpaid
five_star_unpaid = unpaid_program_review.filter(unpaid_program_review.star_rating == 5).count()
five_star_unpaid

411689

In [37]:
# Percentage of 5 star reviews of the paid type
percent_reviews_paid = (five_star_paid / total_paid) * 100
percent_reviews_paid

33.5494666387785

In [38]:
# Percentage of 5 star reviews of the unpaid type
percent_reviews_unpaid = (five_star_unpaid / total_unpaid) * 100
percent_reviews_unpaid

55.92065764559183